In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


# Tensorflow 2.0 GPU 사용 가능 여부 확인

```bash
pip install tensorflow-gpu
# or 
conda install tensorflow-gpu
``` 

텐서플로우버전 및 GPU 사용 가능 여부 확인
```bash
import tensorflow as tf 
print(tf.__version__) 

from tensorflow.python.client import device_lib 
device_lib.list_local_devices()
``` 


In [8]:
import tensorflow as tf 
print(tf.__version__) 

from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

2.4.1


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15119336128902713030]

In [5]:
#  


import tensorflow as tf
# MNIST 데이터셋을 로드 
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 층을 차례대로 쌓아 tf.keras.Sequential 모델을 만듭니다. 
#훈련에 사용할 옵티마이저(optimizer)와 손실 함수를 선택합니다: 
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])
 
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#모델을 훈련하고 평가 
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 1s 435us/step - loss: 0.4799 - accuracy: 0.8568
Epoch 2/5
1875/1875 [==============================] - 1s 436us/step - loss: 0.1513 - accuracy: 0.9542
Epoch 3/5
1875/1875 [==============================] - 1s 439us/step - loss: 0.1089 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 1s 439us/step - loss: 0.0907 - accuracy: 0.9721
Epoch 5/5
1875/1875 [==============================] - 1s 438us/step - loss: 0.0718 - accuracy: 0.9774
313/313 - 0s - loss: 0.0781 - accuracy: 0.9761


[0.07808258384466171, 0.9761000275611877]

In [7]:
import tensorflow as tf
import time
start = time.time()  # 시작 시간 저장

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

# 데이터셋 로드
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis] 

# tf.data 를 사용해 배치 생성 
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


# 케라스(Keras)의 모델 서브클래싱(subclassing) API를 사용 
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel() 

# 옵티마이저와 손실함수 선택
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam() 


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy') 

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
    
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간 

에포크: 1, 손실: 0.1396370381116867, 정확도: 95.88666534423828, 테스트 손실: 0.060036979615688324, 테스트 정확도: 97.97999572753906
에포크: 2, 손실: 0.09087345749139786, 정확도: 97.29000091552734, 테스트 손실: 0.05518272891640663, 테스트 정확도: 98.17500305175781
에포크: 3, 손실: 0.06795856356620789, 정확도: 97.93888854980469, 테스트 손실: 0.05654719099402428, 테스트 정확도: 98.18000030517578
에포크: 4, 손실: 0.05413771793246269, 정확도: 98.34750366210938, 테스트 손실: 0.057880908250808716, 테스트 정확도: 98.20500183105469
에포크: 5, 손실: 0.04491736739873886, 정확도: 98.62633514404297, 테스트 손실: 0.060289572924375534, 테스트 정확도: 98.22999572753906
